# Import Library

In [ ]:
import os
import re
import ast
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import emoji

# Drop Unused Column

In [3]:
file_1_path = r".\Ori_Dataset\ToxicData.csv"
output_file_1 = r".\Ori_Dataset\ToxicData.csv"

df1 = pd.read_csv(file_1_path, encoding='ISO-8859-1', low_memory=False)[['comment_text', 'toxic']]
df1.to_csv(output_file_1, index=False, encoding='ISO-8859-1')

# Rename Column Headers

In [6]:
file_1_path = r".\Ori_Dataset\ToxicData.csv"

output_file_1 = r".\Ori_Dataset\ToxicData.csv"

def dataset (file_path,output_csv_path):
    df = pd.read_csv(file_path, encoding='ISO-8859-1')
    df.rename(columns={'toxic': 'label', 'comment_text': 'text'}, inplace=True)
    df.to_csv(output_csv_path, index=False, encoding='ISO-8859-1')
    print(f"Processed and saved: {output_csv_path}")
dataset (file_1_path,output_file_1)  


Processed and saved: .\Ori_Dataset\ToxicData.csv


# Remove Null

In [8]:
import pandas as pd

file_path = r".\Ori_Dataset\ToxicData.csv"
data = pd.read_csv(file_path, encoding='ISO-8859-1')

print("\nMissing values per column before drop:")
print(data.isnull().sum())
# print(f"\nTotal rows before drop: {len(data)}\n")

# data = data.dropna()

# print(f"Total rows after drop: {len(data)}")

# output_file = r".\Pre_Dataset\1_Combine.csv"
# data.to_csv(output_file, index=False, encoding='ISO-8859-1')



Missing values per column before drop:
text     0
label    0
dtype: int64


# Lowercase

In [4]:
file_path = r".\Ori_Dataset\ToxicData.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

df['text'] = df['text'].str.lower()
print(df['text'].head())

file = r".\Pre_Dataset\1_Lowercasing.csv"
df.to_csv(file, index=False, encoding='ISO-8859-1')

0    explanation\nwhy the edits made under my usern...
1    d'aww! he matches this background colour i'm s...
2    hey man, i'm really not trying to edit war. it...
3    "\nmore\ni can't make any real suggestions on ...
4    you, sir, are my hero. any chance you remember...
Name: text, dtype: object


# Check Duplicated

In [5]:
file_path = r".\Pre_Dataset\1_Lowercasing.csv"
data = pd.read_csv(file_path, encoding='ISO-8859-1')

print("\nMissing values per column before drop:")
print(data.isnull().sum())



Missing values per column before drop:
text     0
label    0
dtype: int64


In [6]:
file_path = r".\Pre_Dataset\1_Lowercasing.csv"
data = pd.read_csv(file_path, encoding='ISO-8859-1')

duplicate_rows = data[data.duplicated()]
print(f"Number of duplicated rows: {duplicate_rows.shape[0]}")

data = data.drop_duplicates()

output_file_path = r".\Pre_Dataset\1_Lowercasing.csv"
data.to_csv(output_file_path, index=False, encoding="utf-8")

duplicate_rows = data[data.duplicated()]
print(f"Number of duplicated rows (after Drop) : {duplicate_rows.shape[0]}")


Number of duplicated rows: 43
Number of duplicated rows (after Drop) : 0


# Remove 

In [ ]:
file_path = r".\Pre_Dataset\1_Lowercasing.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Remove URLs
df['text'] = df['text'].apply(lambda x: re.sub(r'\b(?:http|https|www)\S+|\b\S+\.(com|net|org|io|gov|edu|cn)\b', '', str(x)))

# Remove HTML
html_tags_pattern = r'<.*?>'
# sub(pattern,replace,text)
df['text'] = df['text'].apply(lambda x: re.sub(html_tags_pattern, ' ', str(x)))
df['text'] = df['text'].apply(lambda x: re.sub(r'\S+html\b', ' ', str(x)))

# Remove emojis
def remove_emojis(text):
    text = emoji.demojize(text)
    text = re.sub(r':\w+:', ' ', text)
    return text
df['text'] = df['text'].apply(remove_emojis)

# Remove ASCII characters & delete unused punctuation
df['text'] = df['text'].apply(lambda x: x.replace("_", " "))
df['text'] = df['text'].apply(lambda x: re.sub(r"[^A-Za-z0-9\s']", ' ', str(x)))
df['text'] = df['text'].apply(lambda x: x.replace("'", ""))

# Remove excessive whitespace
# .strip() removes any leading or trailing whitespace from the text
df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Remove only number rows
only_numbers_df = df[df['text'].astype(str).str.strip().str.isdigit()]
# print(only_numbers_df)
df = df[~df['text'].astype(str).str.strip().str.isdigit()] #turn all data into string (astype(str), remove space (strip()), check is whole string is digit)

# Removing elongation (example: goodddddddddd)
df['text'] = df['text'].apply(lambda x: re.sub(r'(.)\1{2,}', r'\1\1', x))

output_file_path = r".\Pre_Dataset\2_remove.csv"
df.to_csv(output_file_path, index=False, encoding='ISO-8859-1')

# Replace Abbreviations

In [8]:
# Replace Abbreviations
file_path = r".\Pre_Dataset\2_remove.csv"
df = pd.read_csv(file_path, encoding='utf-8')

# Abbreviations List base on unknow word diagram
abbreviations = {
    "werent":"were not","arent": "are not",
    "isnt": "is not",
    "cant": "can not",
    "hasnt": "has not",
    "hadnt": "had not",
    "shes": "she is","hes": "he is",
    "youre": "you are", 
    "youll": "you will",
    "youve": "you have",
    "weve": "we have",
    "yall":"you all",
    "theyre": "they are", 
    "theyve": "they have",
    "doesnt": "does not", 
    "dont":"do not",
    "didnt": "did not",
    "wont": "will not",
    "wouldnt": "would not",
    "shouldnt": "should not",
    "couldnt": "could not",
    "im": "i am",
    "iam": "i am",
    "ive": "i have",
    "id": "i would",
    "cuz":"because", "bcuz":"because","becuz":"because",
    "fucksex": "fuck sex",
    "yourselfgo": "yourself go",
    "bitchesfuck": "bitches fuck",
    "youfuck": "you fuck",
    "diff": "different",
    "backgroundcolor": "background color",
}
def replace_abbreviations(text):
    text = str(text)
    for abbr, full_form in abbreviations.items():
        text = re.sub(r'\b' + re.escape(abbr) + r'\b', full_form, text)
    return str(text)
df['text'] = df['text'].apply(replace_abbreviations)

output_file_path = r".\Pre_Dataset\3_Replace_Abbreviations.csv"
df.to_csv(output_file_path, index=False, encoding="utf-8")
print(f"Data has been saved to {output_file_path}")

Data has been saved to .\Pre_Dataset\3_Replace_Abbreviations.csv


# Tokenization

In [ ]:
# Tokenization
file_path = r".\Pre_Dataset\4_Removed_Repeats_word.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

filteredTokens = []
for token in df['text']:
    token = str(token)
    wordtokens = nltk.tokenize.word_tokenize(token)
    filteredTokens.append(wordtokens)
df['text']=filteredTokens

output_file_path = r".\Pre_Dataset\5_Tokenization.csv"
df.to_csv(output_file_path, index=False, encoding="utf-8")

# Remove Stop Word

In [ ]:
# Remove Stop Word
file_path = r".\Pre_Dataset\5_Tokenization.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Convert strings back to lists, stored as strings in the CSV file 
df['text'] = df['text'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) #(Yadav, 2023)

stopTokens = nltk.corpus.stopwords.words("english")

def removeStopWord(words):
    return [word for word in words if word.lower() not in stopTokens]
df['text'] = df['text'].apply(removeStopWord)

output_file_path =r".\Pre_Dataset\6_Remove_StopWord.csv"
df.to_csv(output_file_path, index=False, encoding="utf-8")

# Lemmatization

In [ ]:
file_path = r".\Pre_Dataset\6_Remove_StopWord.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

In [3]:
# Lemmatization
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pandas as pd
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') #used for Part-of-Speech (POS) tagging

lemmatizer = WordNetLemmatizer()

def get_pos_tagging(word):
    #[0][1]:('running', 'VBG')
    #[0][1][0]:('V')
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV} #need this for wordnet cuz wordnet only have 4 postag
    return tag_dict.get(tag, wordnet.NOUN)  # Default to noun if no match

def lemmatize_text(text):
    lemmatized_words = [lemmatizer.lemmatize(word, get_pos_tagging(word)) for word in text] 
    return ' '.join(lemmatized_words)

df['text'] = df['text'].apply(lemmatize_text)

output_file_path = r".\Pre_Dataset\7_Lemmatization.csv"
df.to_csv(output_file_path, index=False, encoding="utf-8")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Final Check

In [4]:
# Check NULL Column
file_path = r".\Pre_Dataset\7_Lemmatization.csv"
data = pd.read_csv(file_path, encoding='ISO-8859-1')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159528 entries, 0 to 159527
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159485 non-null  object
 1   label   159528 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


In [3]:
# check duplicated
file_path = r".\Pre_Dataset\8_Repeated.csv"
data = pd.read_csv(file_path, encoding='ISO-8859-1')

duplicate_rows = data[data.duplicated()]
print(f"Number of duplicated rows: {duplicate_rows.shape[0]}")

Number of duplicated rows: 1309


In [ ]:
file_path = r".\Pre_Dataset\7_Lemmatization.csv"
df = pd.read_csv(file_path, encoding='utf-8')

def remove_excessive_repetition(text, phrase_len=2, repeat=5):
    words = text.split()
    cleaned = []
    i = 0

    while i < len(words):
        phrase = ' '.join(words[i:i + phrase_len])
        count = 1
        j = i + phrase_len

        while j + phrase_len <= len(words) and ' '.join(words[j:j + phrase_len]) == phrase:
            count += 1
            j += phrase_len

        if count >= repeat:
            cleaned.extend(phrase.split())  
            i = j
        else:
            cleaned.append(words[i])
            i += 1

    return ' '.join(cleaned)

df['text'] = df['text'].apply(lambda x: remove_excessive_repetition(x, phrase_len=2, repeat=5))

output_file_path = r".\Pre_Dataset\8_Repeated.csv"
df.to_csv(output_file_path, index=False, encoding="utf-8")


In [ ]:
# drop na and duplicated
file_path = r".\Pre_Dataset\8_Repeated.csv"
df = pd.read_csv(file_path, encoding='utf-8')

def remove_repeated_words(text):
    if isinstance(text, str):
        return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    return text

df['text'] = df['text'].apply(remove_repeated_words)

df = df.dropna()
df = df.drop_duplicates()

output_file_path = r".\Pre_Dataset\Final.csv"
df.to_csv(output_file_path, index=False, encoding="utf-8")